In [1]:
import torch
import torchvision
import numpy as np
from PIL import Image, ImageDraw
!pip install onnx onnxruntime
import onnx
import onnxruntime

In [2]:
#!pip uninstall onnx2torch
#!pip install onnx2torch
!pip install -e git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch

Obtaining onnx2torch from git+https://github.com/KaizerloveRussia/onnx2torch.git#egg=onnx2torch
  Updating ./src/onnx2torch clone
  Running command git fetch -q --tags
  Running command git reset --hard -q ad46e9224f77ccaa06e5a327bc7269aac0778aba
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for onnx2torch (pyproject.toml) ... done
  Created wheel for onnx2torch: filename=onnx2torch-1.5.14-0.editable-py3-none-any.whl size=14406 sha256=df31669a7717570f2132593466c661432147333b12849d0fd38da6aed2a82ffd
  Stored in directory: /tmp/pip-ephem-wheel-cache-hov634yz/wheels/f9/d1/a3/b82562ab7b0402505d5ec31e97cef459ba9bb1251fcc238b54
Successfully built onnx2torch
  Attempting uninstall: onnx2torch
    Found existing installation: onnx2torch 1.5.14
    Uninstalling onnx2torch-1.5.14:
      Successfully uninstalled onnx2

In [3]:
# Load pre-trained faster RCNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
# Load an image
from google.colab import files
uploaded=files.upload()
image = Image.open("image.png").convert('RGB')

Saving image.png to image (3).png


In [5]:
# Preprocess the image
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
image1 = image
image = transform(image)
image = image.unsqueeze(0)

In [6]:
#Run the model and display the results
output=model(image)
print(output)

[{'boxes': tensor([[204.3542,  37.5309, 421.1374, 623.8053],
        [ 19.3879,  73.0953, 223.6135, 511.5916],
        [460.2913, 413.0642, 586.8542, 500.8302],
        [482.1385, 338.9479, 545.5147, 381.8059],
        [251.6486,  46.9328, 382.8713, 262.4328],
        [ 68.5421,  38.3235, 339.7109, 632.1019],
        [146.2069, 124.2338, 224.9680, 474.4522],
        [ 12.2086, 168.0689, 183.7289, 323.8441],
        [270.9537,  44.9405, 333.2106, 184.0173],
        [263.7951, 210.5962, 332.9884, 239.9615],
        [ 10.4931, 108.0453, 188.1809, 339.8682],
        [ 51.1781, 279.1052, 178.2148, 624.9517],
        [208.6113, 125.0314, 376.4835, 347.4301],
        [ 37.6162, 274.2091, 174.9908, 632.8361],
        [296.0182, 206.2048, 331.9722, 243.0341],
        [212.9393,  49.1574, 433.8777, 359.3833],
        [ 94.9690, 370.9078, 167.1342, 635.1591]], grad_fn=<StackBackward0>), 'labels': tensor([ 1,  1, 15,  3,  1,  1,  1,  9, 14, 35,  1,  1, 27, 64, 35,  1, 86]), 'scores': tensor([0.998

In [7]:
# Convert the model to ONNX format
input_names = ["input"]
output_names = ["output"]
torch.onnx.export(model, image, "faster_rcnn.onnx", input_names=input_names, output_names=output_names, opset_version=13)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4009: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (torch.floor((input.size(i + 2).float() * torch.tensor(scale_factors[i], dtype=torch.float32)).float()))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:166: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  boxes_x = torch.min(boxes_x, torch.tensor(width, dtype=boxes.dtype, device=boxes.device))
/usr/local/lib/python3.10/dist-packages/torchvision/ops/boxes.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bo

In [8]:
onnx_model = onnx.load("faster_rcnn.onnx")
ort_session = onnxruntime.InferenceSession("faster_rcnn.onnx")

In [9]:
from onnx2torch.converter import convert
torch_model = convert("faster_rcnn.onnx")

NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='If', version=13))